<a href="https://colab.research.google.com/github/vkuchinski/Imersao_Alura_AI_Risco_Chuva/blob/main/ProjetoAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import geemap
from IPython.display import display, Markdown
import textwrap
from google.colab import userdata

True

In [92]:
# Autenticando o Google Earth Engine (GEE)
ee.Authenticate()

# Abrindo a biblioteca de projetos do GEE
ee.Initialize(project=userdata.get('GEE_PROJECT'))

In [ ]:
# Instalando a API do Google Gen AI
!pip install -q -U google-generativeai

In [35]:
# Importando o SDK do Python para Google Gen AI
import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [62]:
# Função para deixar o texto do modelo mais apresentável para o notebook
def to_markdown(text):
    text = text.replace(' * ', ' > ')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda x: True))

def check_num(text):
  text = text.replace(' * ', ' ')

In [36]:
# Inicializando o modelo
generation_config = {
    'candidate_count': 1,
    'temperature': 0.1,
}

In [37]:
# Escolhendo o modelo
model = genai.GenerativeModel(model_name='gemini-1.0-pro', generation_config=generation_config)

In [38]:
# Abrindo a instância do chat do modelo
chat = model.start_chat(history=[])

In [24]:
# Chuva Média Mensal por Estado do Brasil (mm) (tabela obtida no Gemini)
precipitacao_media_mensal = {
    "Acre": 2120/12,
    "Alagoas": 1100/12,
    "Amazonas": 2820/12,
    "Amapá": 3290/12,
    "Bahia": 1430/12,
    "Ceará": 2470/12,
    "Distrito Federal": 1920/12,
    "Espírito Santo": 1410/12,
    "Goiás": 1900/12,
    "Maranhão": 2470/12,
    "Mato Grosso": 2230/12,
    "Mato Grosso do Sul": 2310/12,
    "Minas Gerais": 1900/12,
    "Pará": 2820/12,
    "Paraíba": 1240/12,
    "Paraná": 1480/12,
    "Pernambuco": 1290/12,
    "Piauí": 1080/12,
    "Rio de Janeiro": 1350/12,
    "Rio Grande do Norte": 1180/12,
    "Rio Grande do Sul": 1530/12,
    "Rondônia": 2200/12,
    "Roraima": 1770/12,
    "Santa Catarina": 1670/12,
    "São Paulo": 1260/12,
    "Sergipe": 1690/12,
    "Tocantins": 1820/12
}

In [106]:
# Definindo as instruções do modelo
instrucao_1 = 'Você é um assistente para obtenção de dados geográficos e de precipitação para a definição de perigo com relação ao volume precipitado em uma área'

instrucao_2 = 'A seguir, segue a tabela de precipitação média anual dos estados do Brasil, posteriormente será requisitada essa informação para fins de comparação com o encontrado. Segue a tabela: ' + str(precipitacao_media_mensal)

instrucao_3 = 'Segundo o INMET, a escala de risco para chuva é a seguinte: Risco amarelo: volume de chuva (em milímetros = mm) de 20 a 30 mm/h ou até 50 mm/dia, Risco laranja: volume de chuva (em milímetros = mm) de 30 a 60 mm/h ou de 50 a 100 mm/dia, Risco vermelho: volume de chuva (em milímetros = mm) maior que 60 mm/h ou maior que 100 mm/dia'

instrucao_4 = 'Ainda segundo o INMET, Amarelo significa perigo potencial, ou seja, os cidadãos devem ter “cuidado na prática de atividades sujeitas a riscos de caráter meteorológico” e para que as pessoas “mantenham-se informadas sobre as condições meteorológicas previstas e não corram risco desnecessário”; Laranja significa perigo e a orientação é para que as pessoas das áreas afetadas “mantenham-se muito vigilantes e informem-se regularmente sobre as condições meteorológicas previstas”. “Inteire-se sobre os riscos que possam ser inevitáveis e siga os conselhos das autoridades”; e Vermelho significa grande perigo, ou seja: “estão previstos fenômenos meteorológicos de intensidade excepcional”, com “grande probabilidade de ocorrência de grandes danos e acidentes com riscos para a integridade física ou mesmo à vida humana”. “Mantenha-se informado sobre as condições meteorológicas previstas e os possíveis riscos. Siga as instruções e conselhos das autoridades em todas as circunstâncias e prepare-se para medidas de emergência”.'

instrucao_5 = 'O resultado final deve ser apresentado na seguinte estrutura: **Local**: Cidade, Estado \n**Data**: Data \n**Análise inicial**: deverá apresentar uma comparação rapida entre o valor da precipitação na data com a média mensal \n**Análise do risco**: apresentar o risco conforme a tabela do INMET, caso haja, e informar quais ações devem ser tomadas pela população \n**Pessoas afetadas**: informar o número de pessoas possivelmente afetadas, baseado na população do município informado'

In [107]:
# Enviando as instruções para o modelo
chat.send_message(instrucao_1)
chat.send_message(instrucao_2)
chat.send_message(instrucao_3)
chat.send_message(instrucao_4)
chat.send_message(instrucao_5)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': '**Estrutura do Resultado Final:**\n\n**Local:** Cidade, Estado\n**Data:** Data\n**Análise Inicial:** Comparação entre o valor da precipitação na data com a média mensal\n**Análise do Risco:** Risco conforme a tabela do INMET (se houver) e ações a serem tomadas pela população\n**Pessoas Afetadas:** Número de pessoas possivelmente afetadas, baseado na população do município informado\n\n**Exemplo:**\n\n**Local:** Santa Maria, RS\n**Data:** 30/04/2024\n**Análise Inicial:** A precipitação registrada em 30/04/2024 foi de 50 mm. A média mensal de precipitação para abril em Santa Maria é de 127,5 mm. Portanto, a precipitação registrada na data foi inferior à média mensal.\n**Análise do Risco:** Risco Amarelo (Perigo Potencial)\n**Ações a serem tomadas pela população:** Tenha cuidado ao praticar atividades sujeitas a riscos meteorológicos.

### Avaliação do perigo relacionado ao volume precipitado no Brasil

In [117]:
cidade = input('Informe a cidade que será a pesquisa\n')
print()
estado = input('Informe o estado que será a pesquisa\n')

local = chat.send_message('O local da pesquisa é na cidade de 'f'{cidade}' 'e no estado 'f'{estado}').text

populacao = chat.send_message('Qual é a população estimada para o local: 'f'{local}').text

Informe a cidade que será a pesquisa
Santa Maria

Informe o estado que será a pesquisa
RS


In [45]:
data = input('Informe o dia da pesquisa no formato DD/MM/AAAA \n')

Informe o dia da pesquisa no formato DD/MM/AAAA 
30/04/2024


In [48]:
# Data no formato do Google Earth Engine
data_ee = ee.Date(data[-4:] + '-' + data[3:5] + '-' + data[0:2])

In [79]:
# Obtendo o valor da chuva média mensal para o estado informado
chuva_media_estado = chat.send_message('Dado a tabela de chuva média mensal por estados, qual é a chuva média mensal para o estado ' f'{estado}' '? Informe SOMENTE o número, em formato string, arredondado para 2 casas decimais com separador decimal dot').text
chuva_media_estado = float(chuva_media_estado.replace('"', ''))

In [85]:
# Obtendo a longitude do local informado
lon = chat.send_message('Com base na cidade de 'f'{cidade}'', localizada no estado do 'f'{estado}'', qual é a longitude do local? Informe SOMENTE o número, em formato string, com separador decimal dot').text
lon = float(lon.replace('"', ''))

In [87]:
# Obtendo a latitude do local informado
lat = chat.send_message('Com base na cidade de 'f'{cidade}'', localizada no estado do 'f'{estado}'', qual é a latitude do local? Informe SOMENTE o número, em formato string, com separador decimal dot').text
lat = float(lat.replace('"', ''))

In [90]:
# Definindo o ponto para obtenção das informações no Google Earth Engine
ponto = ee.Geometry.Point([lon, lat])

In [95]:
# Definindo a bacia hidrográfica do local informado

# Adicionando o título do mapa
display(Markdown("### Mapa de localização da bacia hidrográfica da área de estudo: "f'{cidade}, {estado}'))

# Adicionando o mapa formato geemap
Map = geemap.Map(basemap='Esri.WorldImagery')
Map.add_basemap('CartoDB.PositronOnlyLabels')

# Adicionando as bacias do banco de dados HydroATLAS - HydroSHEDS
bacias = ee.FeatureCollection('WWF/HydroATLAS/v1/Basins/level07')

# Filtrando a bacia para o local informado
bacia_filtrada = bacias.filterBounds(ponto)

# Configurando a visualização da bacia filtrada
style_params = {
  'fillColor': '00000000',  # 100% transparent fill color
  'color': '#FF00FF',  # Red color for the border
  'width': 5  # Thicker border
}

# Definindo a legenda do mapa
legend_dict = {
    'Bacia Hidrográfica': '#FF00FF',
}

# Adicionando a legenda ao mapa
Map.add_legend(legend_dict=legend_dict)

# Adicionando a bacia hidrográfica ao mapa
Map.addLayer(bacia_filtrada.style(**style_params), {}, 'Bacia hidrográfica')

# Centralizando o mapa
Map.centerObject(bacia_filtrada, zoom=10)

# Mostrando o mapa
Map

### Mapa de localização da bacia hidrográfica da área de estudo: Santa Maria, RS

Map(center=[-29.774783823917893, -53.848943163920204], controls=(WidgetControl(options=['position', 'transpare…

In [99]:
# Obtendo informações de precipitação do local para a data informada

# Adicionando o título do mapa
display(Markdown("### Mapa de precipitação acumulada para o dia " f"{data}"))

# Carregando os dados de precipitação do produto Global Precipitaion Measurement
gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').select('precipitationCal')

# Ajustando a data
range = ee.Date(data_ee).getRange('day');
gpm = gpm.filter(ee.Filter.date(range));

# Obtendo o valor de precipitação diária - é dividido por 2 pois o valor é registrado a cada 30 minutos (em mm/h)
total_precipitation = gpm.reduce(ee.Reducer.sum()).divide(2)

# Criando uma máscara para aparecer apenas os pixels em que houve chuva registrada no período
mask = total_precipitation.gt(0.5);
precipitation = total_precipitation.updateMask(mask);

# Criando o mapa
Map = geemap.Map(basemap='CartoDB.DarkMatter')

# Definindo a legenda
legend_dict = {
    '0-30': '#000096',
    '30-60': '#0064ff',
    '60-90': '#00b4ff',
    '90-120': '#33db80',
    '120-150': '#9beb4a',
    '150-180': '#ffeb00',
    '180-210': '#ffb300',
    '210-240': '#ff6400',
    '240-270': '#eb1e00',
    '270-300': '#af0000',
}

# Adicione a legenda ao mapa
Map.add_legend(legend_dict=legend_dict)

# Adiciona o total precipitado no mapa
Map.addLayer(precipitation, {'min': 0, 'max': 300, 'palette': ['000096','0064ff', '00b4ff', '33db80', '9beb4a', 'ffeb00', 'ffb300', 'ff6400', 'eb1e00', 'af0000']}, 'Total Precipitation')

# Adiciona a bacia hidrográfica no mapa
Map.addLayer(bacia_filtrada.style(**style_params), {}, 'Bacia hidrográfica')

# Centralizando o mapa
Map.centerObject(bacia_filtrada, zoom=7)

# Mostrando o mapa
Map

### Mapa de precipitação acumulada para o dia 30/04/2024

Map(center=[-29.774783823917893, -53.848943163920204], controls=(WidgetControl(options=['position', 'transpare…

In [101]:
# Obtendo informações sobre a chuva observada média na bacia hidrográfica
chuva_observada = precipitation.reduceRegion(reducer=ee.Reducer.mean(), geometry=bacia_filtrada, scale=30).getInfo()['precipitationCal_sum']

# Verificação do valor precipitado:
if chuva_observada == None:
  print('Não houve chuva')

else:
  print('Precipitação média da bacia hidrográfica no dia 'f'{data}'':', round(chuva_observada, 2), 'mm')

Precipitação média da bacia hidrográfica no dia 30/04/2024: 263.77 mm


In [130]:
chuva_observada = 10

In [131]:
# Resultado da análise realizada
resultado = chat.send_message('Com base no valor precipitado de 'f'{chuva_observada}'', em 'f'{cidade}'', 'f'{estado}'' no dia 'f'{data}'' que tem uma população estimada de 'f'{populacao}'', qual é o resultado da análise, conforme a instrução repassada anteriormente? Caso o valor da chuva observada seja None, informar que não há dados e não é possível realizar a análise').text

In [132]:
# Apresentando o resultado
display(Markdown('### Análise de risco devido a precipitação'))

display(to_markdown(resultado))

### Análise de risco devido a precipitação

> **Local:** Santa Maria, RS
> **Data:** 30/04/2024
> **Valor da Precipitação:** 10 mm
> **População Estimada:** 285.000 habitantes
> 
> **Análise Inicial:**
> 
> A precipitação registrada em 30/04/2024 foi de 10 mm. A média mensal de precipitação para abril em Santa Maria é de 127,50 mm. Portanto, a precipitação registrada na data foi muito inferior à média mensal.
> 
> **Análise do Risco:**
> 
> Sem risco
> 
> **Ações a serem tomadas pela população:**
> 
> Não há necessidade de tomar ações específicas relacionadas à precipitação.
> 
> **Pessoas Afetadas:**
> 
> Não há pessoas afetadas pelo evento de chuva, pois a precipitação registrada foi muito baixa.
> 
> **Conclusão:**
> 
> O valor da precipitação registrado em 30/04/2024 em Santa Maria, RS, foi muito inferior à média mensal, indicando ausência de risco relacionado à precipitação.